In [72]:
import numpy as np
import tensorflow as tf
FLAGS = None

MAX_DOCUMENT_LENGTH = 10
EMBEDDING_SIZE = 50
n_words = 0
MAX_LABEL = 15
WORDS_FEATURE = 'words'
# Prepare training and testing data  
cancerTotal = pd.merge(cancer_data, cancer_text, on='ID', how='inner')
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(cancerTotal.Text, cancerTotal.Class, test_size=0.2, random_state=42)

  
# Process vocabulary  
vocab_processor = learn.preprocessing.VocabularyProcessor(len(cancerTotal.Text))  
X_train = np.array(list(vocab_processor.fit_transform(X_train))) 
X_test = np.array(list(vocab_processor.transform(X_test)))
n_words = len(vocab_processor.vocabulary_)
print('Total words: %d' % n_words)

Total words: 136444


In [32]:
def bag_of_words_model(features, target):   
    target = tf.one_hot(target, 9, 1, 0)  
    features = tf.contrib.layers.bow_encoder(features, vocab_size=n_words, embed_dim=EMBEDDING_SIZE)  
    logits = tf.contrib.layers.fully_connected(features, 9,activation_fn=None)  
    loss = tf.contrib.losses.softmax_cross_entropy(logits, target)
    train_op = tf.contrib.layers.optimize_loss(loss, tf.contrib.framework.get_global_step(),optimizer='Adam', learning_rate=0.01)  
    return ({'class': tf.argmax(logits, 1), 'prob': tf.nn.softmax(logits)},loss, train_op)

In [33]:
from sklearn.metrics import accuracy_score
classifier = learn.Estimator(model_fn=bag_of_words_model) 
# Train and predict 
classifier.fit(X_train, y_train, steps=500) 
y_predicted = [ p['class'] for p in classifier.predict(X_test, as_iterable=True)] 
score = accuracy_score(y_test, y_predicted) 
print('Accuracy: {0:f}'.format(score))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11be6af98>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/var/folders/vz/p2k6whv90_91t9567mpw6x1h0000gn/T/tmpr3j1ipyp'}
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKC

In [73]:
def rnn_model(features, labels, mode):
  """RNN model to predict from sequence of words to a class."""
  # Convert indexes of words into embeddings.
  # This creates embeddings matrix of [n_words, EMBEDDING_SIZE] and then
  # maps word indexes of the sequence into [batch_size, sequence_length,
  # EMBEDDING_SIZE].
  word_vectors = tf.contrib.layers.embed_sequence(
      features[WORDS_FEATURE], vocab_size=n_words, embed_dim=EMBEDDING_SIZE)

  # Split into list of embedding per word, while removing doc length dim.
  # word_list results to be a list of tensors [batch_size, EMBEDDING_SIZE].
  word_list = tf.unstack(word_vectors, axis=1)

  # Create a Gated Recurrent Unit cell with hidden size of EMBEDDING_SIZE.
  cell = tf.contrib.rnn.GRUCell(EMBEDDING_SIZE)

  # Create an unrolled Recurrent Neural Networks to length of
  # MAX_DOCUMENT_LENGTH and passes word_list as inputs for each unit.
  _, encoding = tf.contrib.rnn.static_rnn(cell, word_list, dtype=tf.float32)

  # Given encoding of RNN, take encoding of last step (e.g hidden size of the
  # neural network of last step) and pass it as features for softmax
  # classification over output classes.
  logits = tf.layers.dense(encoding, MAX_LABEL, activation=None)
  return estimator_spec_for_softmax_classification(
      logits=logits, labels=labels, mode=mode)


In [35]:
import pandas as pd
import numpy as np
from collections import defaultdict
lis_ = defaultdict(list)
import xlrd
cancer_text = pd.read_excel('cancerText.xlsx', sheetname='Sheet2')

cancer_text.columns = ['ID','Text']
cancer_data = pd.read_csv('train_aa_extra.csv')
cancer_data.drop('Unnamed: 0', axis=1, inplace=True)
frames = [cancer_data, cancer_text]
cancerTotal = pd.merge(cancer_data, cancer_text, on='ID', how='inner')
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(cancerTotal.Text, cancerTotal.Class, test_size=0.2, random_state=42)






In [36]:
import os
import sys
import json
import time
import logging
import numpy as np
import tensorflow as tf
from tensorflow.contrib import learn
from sklearn.model_selection import train_test_split



In [78]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

import numpy as np
import pandas
from sklearn import metrics
import tensorflow as tf

FLAGS = None

MAX_DOCUMENT_LENGTH = 10
EMBEDDING_SIZE = 50
n_words = len(vocab_processor.vocabulary_)
MAX_LABEL = 15
WORDS_FEATURE = 'words'  # Name of the input words feature.


def estimator_spec_for_softmax_classification(logits, labels, mode):
    predicted_classes = tf.argmax(logits, 1)
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions={'class': predicted_classes,'prob': tf.nn.softmax(logits)})

    onehot_labels = tf.one_hot(labels, MAX_LABEL, 1, 0)
    loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=logits)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
        train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

        eval_metric_ops = {'accuracy': tf.metrics.accuracy(labels=labels, predictions=predicted_classes)}
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)





def main(unused_argv):
    global n_words


  # Process vocabulary
    vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(
      MAX_DOCUMENT_LENGTH)

    x_transform_train = vocab_processor.fit_transform(X_train)
    x_transform_test = vocab_processor.transform(X_test)

    x_train = np.array(list(x_transform_train))
    x_test = np.array(list(x_transform_test))

    n_words = len(vocab_processor.vocabulary_)
    print('Total words: %d' % n_words)

  # Build model
  # Switch between rnn_model and bag_of_words_model to test different models.
    model_fn = rnn_model
    if FLAGS.bow_model:
    # Subtract 1 because VocabularyProcessor outputs a word-id matrix where word
    # ids start from 1 and 0 means 'no word'. But
    # categorical_column_with_identity assumes 0-based count and uses -1 for
    # missing word.
        x_train -= 1
        x_test -= 1
        model_fn = bag_of_words_model
    classifier = tf.estimator.Estimator(model_fn=rnn_model)

  # Train.
    train_input_fn = tf.estimator.inputs.numpy_input_fn(x={WORDS_FEATURE: x_train},
        y=y_train,
        batch_size=len(x_train),
        num_epochs=None,
        shuffle=True)
    classifier.train(input_fn=train_input_fn, steps=100)

  # Predict.
    test_input_fn = tf.estimator.inputs.numpy_input_fn(x={WORDS_FEATURE: x_test},
      y=y_test,
      num_epochs=1,
      shuffle=False)
    predictions = classifier.predict(input_fn=test_input_fn)
    y_predicted = np.array(list(p['class'] for p in predictions))
    y_predicted = y_predicted.reshape(np.array(y_test).shape)

  # Score with sklearn.
    score = metrics.accuracy_score(y_test, y_predicted)
    print('Accuracy (sklearn): {0:f}'.format(score))

  # Score with tensorflow.
    scores = classifier.evaluate(input_fn=test_input_fn)
    print('Accuracy (tensorflow): {0:f}'.format(scores['accuracy']))



    


TypeError: cannot use a string pattern on a bytes-like object